In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
import torch_classes_v2
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb
import time

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

In [7]:
sum(weights)/len(weights)

0.005000000000000004

In [8]:
len(weights)

200

In [9]:
weights_df = pd.DataFrame(data=list(zip(range(0,201),weights)),columns=['stock_id','index_weight'])

In [10]:
weights_df

,stock_id,index_weight
0,0,0.004
1,1,0.001
2,2,0.002
3,3,0.006
4,4,0.004
...,...,...
195,195,0.004
196,196,0.001
197,197,0.004
198,198,0.006


In [11]:
train = train.merge(weights_df,on='stock_id')

In [12]:
train['wap_calc'] = (train['bid_price']*train['ask_size']+train['ask_price']*train['bid_size'])/(train['ask_size']*train['bid_size'])

In [13]:
# out  

In [14]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc'],
      dtype='object')

In [15]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'wap_t-60'] = df_g['wap'].shift(6)
    df[f'target_t-60'] = df_g['target'].shift(6)
    df[f'initial_wap'] = df_g['wap_calc'].transform('first')
    cols = ['bid_price','ask_price','bid_size','ask_size']
    for i in cols:
        df[f'{i}_t-60'] = df_g[i].shift(-6)

    return(df)

In [16]:
def generate_index(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    # df[f'index_wap_t-60'] = df_g['wap'].shift(6)
    # df[f'index_wap'] = df_g['target'].shift(6)
    df[f'index_wap'] = df_g['wap_weighted'].transform('mean')
    return(df)

In [17]:
def generate_index_2(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'index_wap_t-60'] = df_g['index_wap'].shift(6)
    # df[f'index_wap'] = df_g['target'].shift(6)
    df[f'index_wap_init'] = df_g['index_wap'].transform('first')
    return(df)

In [18]:
def generate_index_3(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'index_wap_t-60'] = df_g['index_wap_move_to_init'].shift(6)
    return(df)

In [26]:
train['index_wap_move_to_init'] = train['index_wap']/train['index_wap_init']

In [21]:
train['wap_weighted'] = train['wap']*train['index_weight']

In [22]:
train['wap_move_to_init'] = train['wap_calc']/train['initial_wap']

In [19]:
train_g = train.groupby(['stock_id','date_id'])
train = generate_prev_race(train,train_g)
train['delta_wap'] = train['wap']/train['wap_t-60']

In [23]:
train_g = train.groupby(['seconds_in_bucket','date_id'])
train = generate_index(train,train_g)
train['delta_wap'] = (train['wap']/train['wap_t-60'])

In [24]:
train_g = train.groupby(['date_id'])
train = generate_index_2(train,train_g)
train['delta_wap'] = (train['wap']/train['wap_t-60'])

In [27]:
train_g = train.groupby(['date_id'])
train = generate_index_3(train,train_g)
train['delta_wap'] = (train['wap']/train['wap_t-60'])

In [28]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc', 'wap_t-60', 'target_t-60', 'initial_wap', 'bid_price_t-60',
       'ask_price_t-60', 'bid_size_t-60', 'ask_size_t-60', 'delta_wap',
       'wap_weighted', 'wap_move_to_init', 'index_wap', 'index_wap_t-60',
       'index_wap_init', 'index_wap_move_to_init'],
      dtype='object')

In [29]:
train['target_calc'] = -((train['wap_t-60']/train['wap'])-(train['index_wap_t-60']/train['index_wap_move_to_init']))*10000
train['target_delta'] = ((train['target_t-60']-train['target_calc'])/train['target_t-60'])*100

In [30]:
train_stock_0 = train[train['stock_id']==0]
train_stock_0.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_size_t-60,delta_wap,wap_weighted,wap_move_to_init,index_wap,index_wap_t-60,index_wap_init,index_wap_move_to_init,target_calc,target_delta
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,...,10085.04,NaN,0.004000,1.000000,0.005031,NaN,0.005031,1.000000,NaN,NaN
1,0,0,10,1299772.70,1,1.000026,15261106.63,NaN,NaN,0.999812,...,17366.82,NaN,0.004000,0.848928,0.005033,NaN,0.005031,1.000356,NaN,NaN
2,0,0,20,1299772.70,1,0.999919,15261106.63,NaN,NaN,0.999812,...,61984.40,NaN,0.003999,2.210527,0.005034,NaN,0.005031,1.000525,NaN,NaN
3,0,0,30,1299772.70,1,1.000133,15261106.63,NaN,NaN,1.000026,...,40433.54,NaN,0.004000,0.294303,0.005034,NaN,0.005031,1.000547,NaN,NaN
4,0,0,40,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000241,...,42572.16,NaN,0.004001,0.788523,0.005035,NaN,0.005031,1.000635,NaN,NaN
5,0,0,50,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000348,...,28375.36,NaN,0.004002,0.930450,0.005035,NaN,0.005031,1.000668,NaN,NaN
6,0,0,60,1218204.43,1,1.000562,15342674.90,NaN,NaN,1.000455,...,68224.23,1.000517,0.004002,1.271280,0.005036,1.000000,0.005031,1.000815,-2.979650,1.652109
7,0,0,70,1264494.89,1,1.000455,15352380.96,NaN,NaN,1.000348,...,13999.50,1.000529,0.004002,0.625777,0.005036,1.000356,0.005031,1.000843,0.420661,-7.913250
8,0,0,80,1189832.86,1,1.000241,15427043.00,NaN,NaN,1.000133,...,25196.40,1.000306,0.004001,0.910560,0.005033,1.000525,0.005031,1.000409,4.216233,0.089475
9,0,0,90,1189272.89,1,1.000562,15427602.97,NaN,NaN,1.000348,...,15769.39,1.000341,0.004002,0.503651,0.005033,1.000547,0.005031,1.000345,5.430774,0.357312


In [107]:
train.to_csv('train_with_new_vars.csv')

In [ ]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id"
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [ ]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [ ]:
y = train['target'].values
X = feat_eng(train)
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [ ]:
lgbm = lgb.Booster(model_file='lgbm_model.lgb')

In [ ]:
lgbm_columns = ['stock_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       'far_price_times_ask_price', 'far_price_minus_bid_price',
       'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb',
       'near_price_minus_ask_price', 'near_price_times_ask_price',
       'near_price_ask_price_imb', 'near_price_minus_bid_price',
       'near_price_times_bid_price', 'near_price_bid_price_imb',
       'near_price_minus_wap', 'near_price_wap_imb',
       'ask_price_minus_bid_price', 'ask_price_times_bid_price',
       'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
       'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
       'reference_price_far_price_near_price_imb2',
       'reference_price_far_price_ask_price_imb2',
       'reference_price_far_price_bid_price_imb2',
       'reference_price_far_price_wap_imb2',
       'reference_price_near_price_ask_price_imb2',
       'reference_price_near_price_bid_price_imb2',
       'reference_price_near_price_wap_imb2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2',
       'far_price_near_price_ask_price_imb2',
       'far_price_near_price_bid_price_imb2', 'far_price_near_price_wap_imb2',
       'far_price_ask_price_bid_price_imb2', 'far_price_ask_price_wap_imb2',
       'far_price_bid_price_wap_imb2', 'near_price_ask_price_bid_price_imb2',
       'near_price_ask_price_wap_imb2', 'near_price_bid_price_wap_imb2',
       'ask_price_bid_price_wap_imb2', 'pca_prices']

In [ ]:
lgbm_preds = lgbm.predict(X[lgbm_columns])

In [ ]:
X['lgbm_preds'] = lgbm_preds
lgbm_preds =  []
del lgbm

In [33]:
X = train

In [34]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

KeyError: "['overall_medvol', 'first5min_medvol', 'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1', 'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid', 'bid_price_over_ask_price', 'reference_price_minus_far_price', 'reference_price_times_far_price', 'reference_price_times_near_price', 'reference_price_minus_ask_price', 'reference_price_times_ask_price', 'reference_price_ask_price_imb', 'reference_price_minus_bid_price', 'reference_price_times_bid_price', 'reference_price_bid_price_imb', 'reference_price_minus_wap', 'reference_price_times_wap', 'reference_price_wap_imb', 'far_price_minus_near_price', 'far_price_times_near_price', 'far_price_minus_ask_price', 'far_price_times_ask_price', 'far_price_minus_bid_price', 'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb', 'near_price_minus_ask_price', 'near_price_times_ask_price', 'near_price_ask_price_imb', 'near_price_minus_bid_price', 'near_price_times_bid_price', 'near_price_bid_price_imb', 'nea

In [32]:
for stock in trading_data.stocksDict.values():
    stock.data_daily = []

train = []

NameError: name 'trading_data' is not defined

In [ ]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

In [ ]:
optim_dict = {
    'RMSProp':optim.RMSprop,
    "Adam":optim.Adam,
    "AdamW":optim.AdamW,
    'SGD':optim.SGD,
    
}

In [ ]:
def model_pipeline(trading_df=trading_data, config=None):
    trading_df = trading_data
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        config = wandb.config
        
        model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')
        config = wandb.config
        optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
        print(model)
        trading_df.reset_hidden(hidden_size=config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [ ]:
importlib.reload(training_testing)

In [ ]:
trading_data.reset_hidden(16,num_layers=5)

In [ ]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]] 
hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [ ]:
hidden_in.shape

In [ ]:
config_static = {'learning_rate':0.001, 'hidden_size':16, 'num_layers':2, 'batch_norm':1}

In [ ]:
config = config_static

In [ ]:
model = model_pipeline(trading_data, config_static)


In [ ]:
model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=4)

In [ ]:
len(targets)

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=1)

stocks_hidden = [torch.stack(x) for x in stocks_hidden]

X = torch.cat(stocks_hidden,dim=-1)

In [ ]:
stocks_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = trading_data.stocksDict[i].lgbm_pred_daily[1]
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        

    stock_hidden.append([torch.cat((x,y.reshape(1)),0) for x,y in zip(trading_data.stocksDict[i].hidden_all,stock_lgbm)])

In [ ]:
stocks_hidden_og = stocks_hidden
X1 = torch.cat(stocks_hidden_og,dim=-1)

In [ ]:
stocks_hidden_og?

In [ ]:
X1?

In [ ]:
X1.shape

In [ ]:
stocks_hidden_og[0] == stocks_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = trading_data.stocksDict[i].lgbm_pred_daily[1]
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        

    stock_hidden.append([torch.cat((x,y.reshape(1)),0) for x,y in zip(trading_data.stocksDict[i].hidden_all,stock_lgbm)])



In [ ]:
stock_hidden[0]

In [ ]:
stock_hidden = []
stock_targets = []

for i in range(0,200):
    # print(i)
    
    try:
        stock_lgbm = torch.stack(trading_data.stocksDict[i].lgbm_pred_daily[1]).reshape(-1,1)
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[1]))
    except KeyError as e:
        stock_targets.append(torch.zeros(55,device='cuda:0'))
        
    # print(stock_lgbm.shape)
    # print(trading_data.stocksDict[i].hidden_all.shape)
    stock_hidden.append([torch.cat((trading_data.stocksDict[i].hidden_all,stock_lgbm),dim=1)])

In [ ]:
stocks_hidden_og = stocks_hidden
X1 = torch.cat(stocks_hidden_og,dim=-1)

In [ ]:
X1?

In [ ]:
stock_lgbm = torch.stack(trading_data.stocksDict[1].lgbm_pred_daily[1])
stock_lgbm.shape

In [ ]:
hidden = trading_data.stocksDict[1].hidden_all
hidden.shape

In [ ]:
torch.cat([hidden,stock_lgbm],dim=1)

In [ ]:
len(stock_targets)

In [ ]:
stocks_hidden = [torch.stack(x) for x in stocks_hidden]
X = torch.cat(stocks_hidden,dim=-1)
Y = torch.stack(targets).transpose(0,1).to('cuda:0')

In [ ]:
output, relu = model(X)

In [ ]:
output.shape

In [ ]:
Y.shape

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "val_epoch_loss", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"values": ["adamW", 'adam', 'SGD', 'RMSprop']},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2,3,4,5]},
    'hidden_size':{'values':[8,16,32,64,128,256,512,1024]},
    'learning_rate': {'max': 0.001, 'min': 0.00001},
    'batch_norm':{'values':[0,1,2]}
}

sweep_config["parameters"] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Optiver Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)